<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/%C3%9Cbungsaufgaben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Einfache Transformationen

In [6]:
# Bibliotheken einmalig installieren

%pip install pyspark pandas

In [7]:
!wget https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py

--2022-05-04 20:28:23--  https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10134 (9.9K) [text/plain]
Saving to: ‘create_data.py.1’

create_data.py.1    100%[===================>]   9.90K  --.-KB/s    in 0s      

2022-05-04 20:28:23 (62.5 MB/s) - ‘create_data.py.1’ saved [10134/10134]



In [8]:
import sqlite3
import pandas as pd
import pyspark.sql.functions as pysparkFunctions 
import os

os.system("python3 create_data.py")

0

In [9]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

(5, 'Barn Ground', 10)
(6, 'Bank', 10)
(7, 'Far Brossler', 20)


Dies ist das erzeugte Dataframe:

    field_id    field_name  area_in_sqm
0         5   Barn Ground           10
1         6          Bank           10
2         7  Far Brossler           20


In [10]:
## (Py)Spark starten

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [11]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(10)


+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
+------------+-----+----+-----------------+-------+-------------+
only showing top 10 rows

+--------+-----+-------+-----------------+----+---

In [12]:
## Zeilen abfragen: Jeremy überlegt welche Pflanze er anbaut und schaut sich dafür die Zucchini genauer an

df_veg_lokal = df_veg.collect()  # eigentlich unnötig, da bei uns ales lokal läuft
print(f"Zucchini in der ersten Woche: {df_veg_lokal[8]}\n")

Zucchini in der ersten Woche: Row(crop='zucchini', field=5, revenue=Row(net=0, tax=0), water_consumption=7, week=5, yield_per_sqm=5)



In [13]:
## Spalten abfragen:	Jeremy will sich Übersicht über generellen Wasserverbrauch der Gemüsesorten verschaffen -> dazu begutachtet er die Spalte 'Wasserverbrauch'  
df_veg.select("crop", "water_consumption").show()

+--------+-----------------+
|    crop|water_consumption|
+--------+-----------------+
|zucchini|                6|
|  radish|                4|
|zucchini|                5|
|  radish|                4|
|zucchini|                6|
|  radish|                5|
|zucchini|                7|
|  radish|                3|
|zucchini|                7|
|  radish|                6|
|zucchini|                9|
|  radish|                7|
|zucchini|               10|
|  radish|                8|
|zucchini|               13|
|  radish|                8|
|zucchini|               12|
|  radish|             null|
|zucchini|                5|
|zucchini|             null|
+--------+-----------------+



In [14]:
## DataFrames Zusammenfügen: Jeremy überlegt ob er Gemüse oder Obst anbauen soll, 
## deshalb verschafft er sich einen Gesamtübersicht/Vergleich aller Obst-&Pflanzensorten: Obst- & Gemüsesorten konkatenieren

## Um die beiden Dataframes kombinieren zu können müssen sie in das gleiche Format gebracht werden.
## Achtung hier verändert sich der semantische Sinn der Spalte. Dies ist hier egal, weil wir sie weiter unten löschen.
df_veg_correct_revenue = df_veg.withColumn("revenue", pysparkFunctions.column("revenue.net"))
df_veg_correct_revenue.show(5)

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

+--------+-----+-------+-----------------+----+-------------+
|    crop|field|revenue|water_consumption|week|yield_per_sqm|
+--------+-----+-------+-----------------+----+-------------+
|zucchini|    5|      0|                6|   1|            0|
|  radish|    6|      0|                4|   1|            0|
|zucchini|    5|      0|                5|   2|            0|
|  radish|    6|      0|                4|   2|            0|
|zucchini|    5|      0|                6|   3|            0|
+--------+-----+-------+-----------------+----+-------------+
only showing top 5 rows

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7| 

In [15]:
## Dupliakte entfernen: Wie Jeremy gerade aufgefallen ist, enthällt die Tabelle noch Duplikate.
## (Siehe z.B. Woche 2 der Erdbeeren) Diese sollen nun entfernt werden.

df_fru_veg = df_fru_veg.dropDuplicates()
df_fru_veg.show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|  10|               10|    100|           25|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   1|               12|      0|            0|
|    zucchini|    5|   0|                7|      5|            5|
|      rad

In [16]:
## Spalten entfernen:   Um einen klareren Blick auf die Daten zu erhalten, entfernt Jeremy alle Spalten
## die nicht über den Wasserverbrauch aussagen

df_fru_veg = df_fru_veg.drop(*('field', 'revenue', 'week', 'yield_per_sqm'))
df_fru_veg.show()

+------------+-----------------+
|        crop|water_consumption|
+------------+-----------------+
|strawberries|               14|
|strawberries|               20|
|strawberries|               24|
|strawberries|               10|
|strawberries|               18|
|strawberries|               26|
|strawberries|             null|
|strawberries|               12|
|strawberries|               10|
|strawberries|               14|
|strawberries|               12|
|    zucchini|                7|
|      radish|                8|
|    zucchini|                6|
|    zucchini|                6|
|    zucchini|                5|
|      radish|                4|
|      radish|             null|
|    zucchini|               12|
|      radish|                6|
+------------+-----------------+
only showing top 20 rows



In [17]:
## Aufräumen:	Für Jeremy sind jetzt natürlich nur Einträge relevant, die einen Wasserverbrauch beinhalten.

df_fru_veg = df_fru_veg.dropna()
df_fru_veg.show()

+------------+-----------------+
|        crop|water_consumption|
+------------+-----------------+
|strawberries|               14|
|strawberries|               20|
|strawberries|               24|
|strawberries|               10|
|strawberries|               18|
|strawberries|               26|
|strawberries|               12|
|strawberries|               10|
|strawberries|               14|
|strawberries|               12|
|    zucchini|                7|
|      radish|                8|
|    zucchini|                6|
|    zucchini|                6|
|    zucchini|                5|
|      radish|                4|
|    zucchini|               12|
|      radish|                6|
|      radish|                3|
|    zucchini|               13|
+------------+-----------------+
only showing top 20 rows



In [18]:
## Wasserverbrauch anzeigen:    Jeremy möchte wissen, wie der gesamte Wasserverbrauch im letzten Jahr war.

df_fru_veg.agg(pysparkFunctions.sum("water_consumption")).collect()

[Row(sum(water_consumption)=285)]

In [19]:
## Wasserverbrauch pro Pfanzenart anzeigen: Jeremy möchte noch wissen, wie der Wasserverbrauch für die unterschiedlichen Gewächsarten ist.

df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(pysparkFunctions.sum("water_consumption")).collect()

[Row(sum(water_consumption)=160)]

In [21]:
## Daten speichern: Um langfristig auf die Daten zurückgreifen zu können, speichern wird jetzt noch die Rohdaten in einer Datenbank.

connection = sqlite3.connect('result.sqlite')

df_raw_data = df_fru.union(df_veg_correct_revenue).drop_duplicates()
df_raw_data.toPandas().to_sql('water_consumption', connection, if_exists='replace', index=True)

print(connection.execute('SELECT * FROM water_consumption;').fetchall())

connection.close()


[(0, 'strawberries', 7, 5, 14.0, 0, 5), (1, 'strawberries', 7, 7, 20.0, 60, 25), (2, 'strawberries', 7, 9, 24.0, 150, 25), (3, 'strawberries', 7, 2, 10.0, 0, 0), (4, 'strawberries', 7, 6, 18.0, 30, 10), (5, 'strawberries', 7, 8, 26.0, 150, 25), (6, 'strawberries', 7, 11, None, 150, 0), (7, 'strawberries', 7, 3, 12.0, 0, 0), (8, 'strawberries', 7, 10, 10.0, 100, 25), (9, 'strawberries', 7, 4, 14.0, 0, 0), (10, 'strawberries', 7, 1, 12.0, 0, 0), (11, 'zucchini', 5, 0, 7.0, 5, 5), (12, 'radish', 6, 50, 8.0, 8, 240), (13, 'zucchini', 5, 0, 6.0, 1, 0), (14, 'zucchini', 5, 0, 6.0, 3, 0), (15, 'zucchini', 5, 0, 5.0, 2, 0), (16, 'radish', 6, 0, 4.0, 1, 0), (17, 'radish', 6, 50, None, 1, 0), (18, 'zucchini', 5, 40, 12.0, 9, 25), (19, 'radish', 6, 0, 6.0, 5, 0), (20, 'radish', 6, 0, 3.0, 4, 0), (21, 'zucchini', 5, 40, 13.0, 8, 25), (22, 'radish', 6, 0, 4.0, 2, 0), (23, 'zucchini', 5, 40, None, 11, 0), (24, 'zucchini', 5, 16, 10.0, 7, 25), (25, 'zucchini', 5, 40, 5.0, 10, 25), (26, 'radish', 6, 0

**Übungen**

1. Dieses Notebook enthält noch eine weitere JSON Datei namens `Corn.json` mit Daten zu Gedreide. Werte den Wasserverbrauch nur für das Gedreide aus.
2. Ermittle wie viel Wasser pro Feld eingesetzt wurde.
3. Ermittle den gesamten Wasserverbrauch für alle Pfanzen.